In [25]:
import pandas as pd
import json
import csv
import numpy as np
import psycopg2
import ast
from sqlalchemy import create_engine, text

In [26]:
pd.set_option('display.max_columns', None)
compounds_df = pd.read_csv('homePage_data.csv')
compounds_df.head()

,id,compound_name,is_super,slug,properties_count,image_url,min_price,max_price,min_unit_area,max_unit_area,available_bathrooms,available_bedrooms,developer_id,developer_name,developer_logo,area_id,area_name,sponsored,max_installment_years,min_down_payment,min_ready_by,properties_ids,sum_10_properties_min_price,lat,long,has_offer,has_launches,is_launch,offer_title,limited_time_offer,property_types,property_types_count
0,195,Mountain View ICity New Cairo,True,195-mountain-view-icity-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,7584822,14559080,130,295,[3],[3],6,DMG,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,758483.0,2027-01-01,"[35486, 35487, 35489, 35490, 35495, 35497, 35502]",117896389.0,30.065182,31.593508,False,False,False,"5% Down Payment , instalments up to 9 years",True,"['Apartment', 'Villa', 'Townhouse', 'Duplex', ...","[{'id': 21, 'name': 'Apartment', 'count': 41, ..."
1,274,ZED,True,274-zed-el-sheikh-zayed-ora,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,10273202,20284992,160,301,"[3, 4]","[3, 4]",85,Ora Developers,https://s3.eu-central-1.amazonaws.com/prod.ima...,26,El Sheikh Zayed,0,8.0,513661.0,2024-10-31,"[41998, 41999, 42002, 42110, 42111, 42217, 42528]",129709414.0,30.046608,31.003101,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Penthouse', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 10, ..."
2,202,Bloomfields,True,202-bloomfields-tatweer-misr-mostakbal-city,76,https://s3.eu-central-1.amazonaws.com/prod.ima...,3664000,16210000,70,287,"[1, 2, 3, 4]","[1, 2, 3, 4]",33,Tatweer Misr,https://s3.eu-central-1.amazonaws.com/prod.ima...,10,Mostakbal City,0,8.0,209350.0,2023-12-30,"[17436, 31836, 31839, 31844, 31847, 31848, 318...",35147621.0,30.070208,31.649590,False,False,False,NaN,True,"['Apartment', 'Duplex', 'Studio']","[{'id': 21, 'name': 'Apartment', 'count': 76, ..."
3,223,Badya,True,223-badya-palm-hills,42,https://s3.eu-central-1.amazonaws.com/prod.ima...,4782600,31084000,107,499,"[2, 3, 4, 6, 7, 8]","[2, 3, 4, 5, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,1,6th of October City,0,10.0,0.0,2023-11-29,"[9742, 11180, 13044, 16026, 19171, 19199, 1961...",44446502.0,29.854789,30.907116,False,False,False,NaN,True,"['Apartment', 'Villa', 'Townhouse', 'Penthouse...","[{'id': 21, 'name': 'Apartment', 'count': 48, ..."
4,148,Palm Hills New Cairo,True,148-palm-hills-new-cairo,7,https://s3.eu-central-1.amazonaws.com/prod.ima...,18886000,72968000,190,611,"[4, 7, 8]","[3, 4, 6]",16,Palm Hills Developments,https://s3.eu-central-1.amazonaws.com/prod.ima...,2,New Cairo,0,9.0,944300.0,2022-12-30,"[22676, 23315, 28769, 28770, 32932, 37716, 42215]",487964000.0,30.059658,31.598640,False,False,False,NaN,True,"['Apartment', 'Villa', 'Twinhouse', 'Townhouse']","[{'id': 21, 'name': 'Apartment', 'count': 13, ..."


In [27]:
# handle the dataframe to be like db table
# Assuming you have an existing DataFrame 'df'
propert_types_table = pd.DataFrame()

# Assuming compounds_df is another DataFrame with 'id' and 'property_types_count' columns

for compoud_id in compounds_df['id']:
    property_types_count = compounds_df[compounds_df['id'] == compoud_id]['property_types_count']
    #print(compoud_id)
    
    # Convert the string to a list of dictionaries
    property_types_count = ast.literal_eval(property_types_count.iloc[0])
    
    if not propert_types_table.empty:
        # Append new values to the existing DataFrame
        new_df = pd.DataFrame(property_types_count)
        new_df['compound_id'] = compoud_id
        propert_types_table = pd.concat([propert_types_table, new_df], ignore_index=True)
    else:
        # Create a new DataFrame if 'df' is empty
        propert_types_table = pd.DataFrame(property_types_count)
        propert_types_table['compound_id'] = compoud_id

# Display the resulting DataFrame
print(propert_types_table.head())

     id       name  count  total  rank  compound_id
0  21.0  Apartment   41.0   41.0   1.0          195
1   1.0      Villa    6.0    6.0   2.0          195
2   7.0  Townhouse    2.0    2.0   4.0          195
3   3.0     Duplex   14.0   14.0   5.0          195
4   4.0  Penthouse    1.0    1.0   6.0          195


In [28]:
# connnect to db 
# Replace these with your database details
dbname = "nawy"
user = "nady"
password = "admin"
host = "172.20.0.3"
port = "5432"


# Establish a connection to the database
try:
    connection = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

except psycopg2.Error as e:
    print("Error connecting to the database:", e)
    exit()


# Create SQLalchemy engine
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

In [31]:
#check df has the same db tables column name

# Query to retrieve column names
with engine.connect() as connection:
    query = text("SELECT column_name FROM information_schema.columns WHERE table_name = 'property_types_count' ORDER BY column_name ASC;")
    results = connection.execute(query)

    # Fetch the results
    column_names = results.fetchall()
    
# check matching betwween df columns vs db columns
columns = sorted(propert_types_table.columns)
for row, column_name in enumerate(column_names):
    results = column_name[0] == columns[row]
    if results == True:
        pass
    else:
        print(f"{column_name[0]} vs {columns[row]}")

In [30]:
### rename propert_types_table column to match the db coloumn names
column_matching = {'count': 'count_',
                   'id': 'property_id'}
propert_types_table.rename(columns=column_matching, inplace=True)
propert_types_table.head()

,property_id,name,count_,total,rank,compound_id
0,21.0,Apartment,41.0,41.0,1.0,195
1,1.0,Villa,6.0,6.0,2.0,195
2,7.0,Townhouse,2.0,2.0,4.0,195
3,3.0,Duplex,14.0,14.0,5.0,195
4,4.0,Penthouse,1.0,1.0,6.0,195


In [32]:
## match column datatypes in db table with column datatypes in df table
column_types = propert_types_table.dtypes.sort_index()
with engine.connect() as connection:
    query = text(
        f"select column_name, data_type from information_schema.columns where table_name='property_types_count' ORDER BY column_name"
    )
    # Excute the query
    results = connection.execute(query)

    #fetch the result
    column_info = results.fetchall()
for row, column_ in enumerate(column_info):
    #print(column_[0])
    results = column_[0] == columns[row]
    if results == True:
        print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")
    else:
        print(f"{column_[0]} vs {columns[row]}")

compound_id                    --> integer             , :: int64
count_                         --> integer             , :: float64
name                           --> character varying   , :: object
property_id                    --> integer             , :: float64
rank                           --> integer             , :: float64
total                          --> integer             , :: float64


/tmp/ipykernel_34970/180780802.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{column_[0]:<30} --> {column_[1]:<20}, :: {column_types[row]}")


In [33]:
# casting all foat df column to integer
propert_types_table['property_id'] = propert_types_table['property_id'].astype(int)
propert_types_table['count_'] = propert_types_table['count_'].astype(int)
propert_types_table['total'] = propert_types_table['total'].astype(int)
propert_types_table['rank'] = propert_types_table['rank'].astype(int)

In [34]:
propert_types_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  1018 non-null   int64 
 1   name         1018 non-null   object
 2   count_       1018 non-null   int64 
 3   total        1018 non-null   int64 
 4   rank         1018 non-null   int64 
 5   compound_id  1018 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 47.8+ KB


In [35]:
propert_types_table.head()

,property_id,name,count_,total,rank,compound_id
0,21,Apartment,41,41,1,195
1,1,Villa,6,6,2,195
2,7,Townhouse,2,2,4,195
3,3,Duplex,14,14,5,195
4,4,Penthouse,1,1,6,195


In [38]:
# insert data to database
propert_types_table.to_sql('property_types_count', engine, if_exists='append', index=False)

18